# Tenacity - Israel Zúñiga de la Mora

## Python Monterrey: Primera reunión 2019 
### 22/Febrero/2019 07:00 P.M. @ WeWork (Blvd. Antonio L. Rodriguez 1888)

![Pymty](py-mty-logo.png)

# `whois(me)`

Personal:

* 89
* Colima, Col. 🇲🇽

Uni|Grado|Años
------|---------|----
Universidad de Colima|Ing. en Telemática|2008-2012
Universidad Abierta y a Distancia de México (UnADM)|Lic. en Matemáticas|2018-∞
Collective Academy|Master of Business & Technology|2019-2020
Universidad de la Calle|Perreología|?
----

Profesional:
* UDC
* TecLID A.C. ✝
* Intel Corp.
* ZapopanLab
* Invictus.mx
* Alturin ✝
* Banregio
* Evolve Fintech / Ictineo PTF
----
* https://github.com/israelzuniga  >>> https://github.com/israelzuniga/slides  (para encontrar esta presentación)  
* https://www.linkedin.com/in/israelzuniga/
* https://twitter.com/0xD1

# Colabora y participa con la comunidad!

## Framewoks, bibliotecas de código, módulos, patrones, challenges, pypy, cpython, threading, Uc, etc

## https://www.facebook.com/groups/pymty/  | https://www.meetup.com/pythonmty/

# Designing for Failure

> Exception handling is one of the most brushed aside aspects of computer programming. Errors are complicated to handle, and often they are unlikely, so developers always forget to handle failures... sometimes they even forgot on purpose.

> However, in a world where applications are distributed over the network, across miles of fiber optic cable and on different computers, failure is not an exception. It must be considered as the norm for your software. Failure scenarios must be first-citizens of the various testing scenarios being developed.

> In an environment distributed over a network, anything that can fail **will** fail.

> Python does not offer any help in that regard, and almost no programming language offers advanced error recovery or retryng capability -- except mabe languages implementing [condition systems](https://en.wikipedia.org/wiki/Exception_handling#Condition_systems) such as Common Lisp.

-- Julien Danjou - The Hacker's Guide to Scaling Python (2017)

# Retry pattern / Retrying pattern


Habilita una aplicación para conducir fallas transitorias cuando intenta conectarse a un servicio o recurso de red, volviendo a intentar de forma transparente la operación fallida. Pudiendo mejorar la estabilidad de la aplicación.





[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)



## Contexto y Problema
Una aplicación que se comunica con elementos corriendo en la nube, debe ser sensitiva a fallas transitorias que pueden ocurrir en este entorno. Las fallas incluyen la pérdida momentánea de conectividad de la red a los componentes y servicios, la indisponibilidad temporal de un servicio o los tiempos de espera que se producen cuando un servicio está ocupado.

Estas fallas suelen ser autocorregibles, y si la acción que desencadenó una falla se repite después de un retraso adecuado, es probable que tenga éxito. 

## Soluciones
* Cancelar
* Volver a procesar (retry)
* Volver a procesar después de un retraso (retry after delay)

[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

![](https://docs.microsoft.com/en-us/azure/architecture/patterns/_images/retry-pattern.png)



[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# Problemas y consideraciones

* Ajustar la política de reintentos según los requisitios de la aplicación. Para algunas operaciones no críticas, es mejor fallar rápido en lugar de volver a intentarlo varias veces e impactar el rendimiento de la aplicación.

* Una política agresiva de reintento con un retraso mínimo entre intentos y un gran número de reintentos podría degradar aún más un servicio ocupado que se está ejecutando cerca o en su capacidad. 
Debe considerar los siguientes puntos al decidir cómo implementar este patrón.

* Si la lógica de negocio lo permite, optar por  reportar o guardar la excepción y pasar a un recurso más degradado. Esto para mejorar la estabilidad y resilencia de la aplicación. Según el patrón [Circuit Breaker](https://docs.microsoft.com/en-us/azure/architecture/patterns/circuit-breaker)


[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# Cuando NO usar este patrón

* Cuando es probable que una falla sea duradera: La aplicación podría estar desperdiciando tiempo y recursos tratando de repetir una solicitud que es probable que falle.
   
* Para el manejo de fallas que no se deben a fallas transitorias, como las excepciones internas causadas por errores en la lógica de negocios de una aplicación.
 
* Como alternativa a la solución de problemas de escalabilidad en un sistema. Si una aplicación experimenta fallos frecuentes y ocupados, a menudo es una señal de que el servicio o recurso al que se accede debe ampliarse.



[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# Cuando  SI usar este patrón

* Cuando una aplicación podría experimentar fallas transitorias cuando interactúa con un servicio remoto o accede a un recurso remoto.

* Cuando los fallos sean de corta duración, y la repetición de una solicitud que previamente haya fallado podría tener éxito en un intento posterior.

[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# ¿Por que te debería importar?

Añadir un manejo inteligente de excepciones permite a tu aplicación/codigo falle correctamente.

# Casos de uso
* Web scraping
* Conexiones a servidores remotos (p. ej. ssh+paramiko, DBs, confirmaciones de transferencia de información)
* Carga/Descarga de archivos
* Código que pueda fallarCode that can fail


# Naive Retrying

## Retrying pattern


In [53]:
import time
import random
from datetime import datetime

def do_something():
    cur_time = datetime.now().strftime('%H:%M:%S:%f')
    if random.randint(0,1) == 0:
        print('{0}: Failure'.format(cur_time))
        raise RuntimeError
    print('{0}: Success'.format(cur_time))

In [55]:
while True:
    try:
        do_something()
    except:
        pass
    else:
        break

22:37:42:028459: Failure
22:37:42:032881: Failure
22:37:42:032997: Failure
22:37:42:033089: Success


## Retrying pattern w/sleep
Espera un número fijo de segundos antes de volver a procesar.


In [57]:
 while True:
    try:
        do_something()
    except:
        # Nos dormimos mamalon por 5 seg antes de retry
        time.sleep(5)
    else:
        break
        

22:37:47:271282: Failure
22:37:52:272306: Success


## Retrying pattern w/exponential backoff
Espera con base en un retroceso exponencial (1 seg  -> 2 -> 4 -> 8 -> 16 -> etc)

In [60]:
attempt = 0
while True:
    try:
        do_something()
    except:
        # Dormimos 2^attemp seg antes de reintentar
        time.sleep(2 ** attempt)
        attempt += 1
    else:
        break

22:38:26:508438: Failure
22:38:27:511175: Failure
22:38:29:515638: Failure
22:38:33:519442: Failure
22:38:41:522556: Success


# Meet: Tenacity
Tenacity es una biblioteca para Python 2 y 3, diseñada para reintentar la ejecución de una tarea o proceso cuando ocurre una `Exception`. Es un fork de [retry](https://github.com/rholder/retrying). El mantenedor es [Julien Danjou](https://julien.danjou.info/).

Repo: https://github.com/jd/tenacity

## Features
* [Funcionalidad de `Decorator`](https://www.thecodeship.com/patterns/guide-to-python-function-decorators/)
* Especificación de condiciones:
    * tiempo en espera/wait (
    * cancelación
* Es
